In [24]:
import tensorflow as tf
import os

# Create a list of all the image files in the folder
image_files = os.listdir('/Users/aayushrangra/Desktop/ObjectDetection/cancer')

# Create a TensorFlow dataset from the list of image files
dataset = tf.data.Dataset.from_tensor_slices(image_files)

# Preprocess the images in the dataset
def preprocess_image(image):
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, (224, 224))
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    return image

dataset = dataset.map(preprocess_image)

# Batch the images in the dataset
dataset = dataset.batch(32)


In [25]:


class YOLOv7(tf.keras.Model):
    def __init__(self, num_classes):
        super(YOLOv7, self).__init__()

        # Backbone network
        self.backbone = tf.keras.applications.ResNet50(
            input_shape=(416, 416, 3),
            include_top=False,
            weights='imagenet'
        )

        # Neck network
        self.neck = tf.keras.Sequential([
            tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        ])

        # Head network
        self.head = tf.keras.Sequential([
            tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(num_classes * 85, (1, 1), activation='linear'),
        ])

    def forward(self, x):
        # Backbone features
        x = self.backbone(x)

        # Neck features
        x = self.neck(x)

        # Head features
        x = self.head(x)

        # Reshape the output to (B, H, W, 85 * C)
        x = tf.reshape(x, (-1, x.shape[1], x.shape[2], 85 * num_classes))

        return x

# Create a YOLOv7 model
model = YOLOv7(num_classes=80)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

import tensorflow as tf

class YOLOv7(tf.keras.Model):
    def __init__(self, num_classes):
        super(YOLOv7, self).__init__()

        # Backbone network
        self.backbone = tf.keras.applications.ResNet50(
            input_shape=(416, 416, 3),
            include_top=False,
            weights='imagenet'
        )

        # Neck network
        self.neck = tf.keras.Sequential([
            tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        ])

        # Head network
        self.head = tf.keras.Sequential([
            tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(num_classes * 85, (1, 1), activation='linear'),
        ])

    def call(self, x):
        # Backbone features
        x = self.backbone(x)

        # Neck features
        x = self.neck(x)

        # Head features
        x = self.head(x)

        # Reshape the output to (B, H, W, 85 * C)
        x = tf.reshape(x, (-1, x.shape[1], x.shape[2], 85 * num_classes))

        return x
    
# Create a YOLOv7 model
model = YOLOv7(num_classes=80)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=10)





Epoch 1/10


ValueError: in user code:

    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/8n/95vlvnl93lg2r18pnsrgf8f40000gn/T/__autograph_generated_fileqhrjk2g0.py", line 10, in tf__call
        x

In [4]:

from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, Dense, Conv2D, BatchNormalization, Activation, Multiply, Add

class RCBAM(Layer):
    def __init__(self, channels):
        super(RCBAM, self).__init__()

        # Channel Attention Module (CAM)
        self.avg_pool = GlobalAveragePooling2D()
        self.max_pool = Conv2D(channels // 16, (1, 1), padding='same')
        self.channel_attention = Dense(channels, activation='sigmoid')

        # Spatial Attention Module (SAM)
        self.conv1 = Conv2D(channels // 16, (1, 1), padding='same')
        self.conv2 = Conv2D(channels // 16, (7, 7), padding='same')
        self.spatial_attention = Dense(channels, activation='sigmoid')

    def call(self, inputs):
        # Channel Attention Module (CAM)
        avg_pool = self.avg_pool(inputs)
        max_pool = self.max_pool(inputs)
        channel_attention = self.channel_attention(avg_pool + max_pool)

        # Spatial Attention Module (SAM)
        avg_branch = self.conv1(inputs)
        max_branch = self.conv2(inputs)
        spatial_attention = self.spatial_attention(tf.concat([avg_branch, max_branch], axis=-1))

        # RCBAM
        channel_attention = tf.expand_dims(channel_attention, axis=1)
        spatial_attention = tf.expand_dims(spatial_attention, axis=-1)
        attention = Multiply()([channel_attention, spatial_attention])
        output = Multiply()([inputs, attention])
        output = Add()([inputs, output])

        return output



In [5]:
class PAM(tf.keras.Model):
    def __init__(self, channels):
        super(PAM, self).__init__()

        self.channel_attention = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(channels // 16, activation='relu'),
            tf.keras.layers.Dense(channels, activation='sigmoid')
        ])

        self.spatial_attention = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=1, kernel_size=(7, 7), padding='same', activation='relu'),
            tf.keras.layers.Conv2D(filters=channels, kernel_size=(7, 7), padding='same', activation='sigmoid')
        ])

    def call(self, inputs):
        channel_attention = self.channel_attention(inputs)
        spatial_attention = self.spatial_attention(inputs)

        attention = channel_attention * spatial_attention

        # Broadcast the attention tensor to the same shape as the inputs tensor
        attention = tf.broadcast_to(attention, tf.shape(inputs))

        return inputs * attention

In [6]:
# Create the SSD model
resnet = tf.keras.applications.resnet50.ResNet50(
    input_shape=(224, 224, 3), 
    include_top=False,
    weights='imagenet'
)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(224, 224, 3)))
model.add(resnet)
# Add the RCBAM attention mechanism

2023-10-08 05:38:09.058550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [10]:
model.add(RCBAM)

# Add the following line:
model.add(tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 7, 7, 2048))))
model.add(PAM(channels=2048))




# Add a dilated convolution layer to the end of the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), dilation_rate=2, activation='relu', input_shape=(64, 64, 3)),
])

TypeError: The added layer must be an instance of class Layer. Received: layer=<class '__main__.RCBAM'> of type <class 'type'>.

In [7]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 60, 60, 64)        1792      
                                                                 
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


In [8]:


# Add a flatten layer
model.add(tf.keras.layers.Flatten())



# Add a dense layer with 2 output units
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [9]:

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=10)

# Save the model
model.save('brain_tumor_model.h5')

# Load the model
model = tf.keras.models.load_model('brain_tumor_model.h5')


Epoch 1/10


ValueError: in user code:

    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/engine/input_spec.py", line 228, in assert_input_compatibilit

In [ ]:

# Load the input image
img = tf.io.decode_jpeg(tf.io.read_file('brain_tumor_image.jpg'))

# Preprocess the image
img = tf.image.convert_image_dtype(img, tf.float32)
img = tf.subtract(img, 0.5)
img = tf.multiply(img, 2.0)

# Detect objects in the image
detections = model(img)


In [ ]:

# Visualize the detections
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.imshow(img)

for detection in detections:
    bbox = detection['bbox'].decode('utf-8')
    ax.add_patch(tf.keras.preprocessing.image.Rectangle(
    bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1], color='red'
))

ax.set_title('Brain tumor detection using SSD with ResNet-50 backbone, RCBAM, PAM and dilated convolution')
plt.show()

NameError: name 'nvidia' is not defined

In [5]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("/Users/aayushrangra/Desktop/ObjectDetection/LeafDisease/Grape Esca Black Measles/Grape_Esca_Black_Measles8465.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:])
print("Confidence Score:", confidence_score)


1/1 [==============================] - 1s 625ms/step
Class: espa

Confidence Score: 1.0


In [3]:
import torch
from torch import nn
from torchvision import models

class EfficientNetMG(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetMG, self).__init__()

        # Load the EfficientNet-MG pre-trained weights from ImageNet
        efficientnet = models.efficientnet_mg(pretrained=True)


        # Remove the last classification layer
        efficientnet.classifier = nn.Identity()

        # Add a new multi-scale feature fusion (MSFF) module
        self.msff = nn.Sequential(
            nn.Conv2d(in_channels=2048, out_channels=1280, kernel_size=1, padding=0),
            nn.BatchNorm2d(1280),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=1280, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        # Add the final classification layer
        self.classifier = nn.Linear(256, 10)

    def forward(self, x):
        # Pass the input image through the EfficientNet-MG model
        x = efficientnet(x)

        # Pass the output features through the MSFF module
        x = self.msff(x)

        # Flatten the features and pass them through the final classification layer
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
        x = self.classifier(x)

        return x

# Example usage:

model = EfficientNetMG(pretrained=True)

# Load the image
image = torch.randn(1, 3, 224, 224)

# Pass the image through the model
output = model(image)

# Get the predicted class
predicted_class = torch.argmax(output, dim=1)

# Print the predicted class
print(predicted_class)



ModuleNotFoundError: No module named 'torch._custom_ops'

In [1]:
import os
import cv2  # or from PIL import Image


In [4]:

# List of directories with images
directories = ['Grape Black rot', 'Grape Esca Black Measles', 'Grape healthy', 'Grape Leaf blight Isariopsis Leaf Spot']

# Preprocessing steps
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image -= [0.485, 0.456, 0.406]
    image /= [0.229, 0.224, 0.225]
    return image

# Loop through each directory
for directory in directories:
    directory_path = os.path.join('/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/LeafDisease', directory)
    
    # Create a new directory for preprocessed images
    output_directory = os.path.join('preprocessed_images', directory)
    os.makedirs(output_directory, exist_ok=True)

    # List image files in the current directory
    image_files = [f for f in os.listdir(directory_path) if f.endswith(('.jpg', '.png', '.jpeg', '.gif'))]

    # Process and save images
    for filename in image_files:
        image_path = os.path.join(directory_path, filename)
        image = cv2.imread(image_path)  # or Image.open(image_path) if using PIL
        preprocessed_image = preprocess_image(image)
        
        output_path = os.path.join(output_directory, 'preprocessed_' + filename)
        cv2.imwrite(output_path, preprocessed_image)  # or preprocessed_image.save(output_path) if using PIL
        if image is not None:
            preprocessed_image = preprocess_image(image)
            
            output_path = os.path.join(output_directory, 'preprocessed_' + filename)
            cv2.imwrite(output_path, (preprocessed_image * 255).astype('uint8'))
